In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score

In [2]:
df = pd.read_excel("C:\\Users\\PC\\Desktop\\Case's\\Garanti BBVA\\Case_Data 3-4-5 _v2.xlsx")

In [3]:
df.head()

,Dummy_id,EAD,Kur_bilgisi,Segment,Default_Flag,Product_Type,BET_Score,Bank_Score
0,4700,1399.355804,TRL,BA,0,5,1.0256,6.9583
1,3309,489.412761,TRL,BB,0,2,1.0481,3.2632
2,1333,6215.708466,TRL,SB,0,.,1.0911,9.9942
3,4920,311.369747,TRL,BA,0,3,1.0951,3.1281
4,4943,554.593179,TRL,BA,0,3,1.1932,4.7455


In [4]:
df.loc[df['Kur_bilgisi'] == 'USD', 'EAD'] *= 20.5
df.loc[df['Kur_bilgisi'] == 'USD', 'Kur_bilgisi'] = 'TRL'

df.loc[df['Kur_bilgisi'] == 'EUR', 'EAD'] *= 22.8
df.loc[df['Kur_bilgisi'] == 'EUR', 'Kur_bilgisi'] = 'TRL'

In [5]:
df = df[(df['Default_Flag'] != 2) & (df['Default_Flag'] != 99)]

In [6]:
df['Product_Type'] = df['Product_Type'].replace('.', np.nan)
df['BET_Score'] = df['BET_Score'].replace('.', np.nan)

In [7]:
bet_df = df[df['BET_Score'].notna()]

In [8]:
# Adım 1: BET_Score'u boş olmayan gözlemlerle sınırlama
betscore_df = df.dropna(subset=['BET_Score'])

# Genel olarak Bet ve Banka skor kartının performansını karşılaştırma
gini_betscore = roc_auc_score(betscore_df['Default_Flag'], betscore_df['BET_Score'])
gini_bankscore = roc_auc_score(df['Default_Flag'], df['Bank_Score'])

print("Genel BET Skor Kartının Performansı (Gini Katsayısı):", gini_betscore)
print("Genel Banka Skor Kartının Performansı (Gini Katsayısı):", gini_bankscore)
print("----------------------------------------")

#İki skor arasındaki korelasyonu inceleme
correlation = df['Bank_Score'].corr(betscore_df['BET_Score'])
print("Banka Skoru ve BET Skoru Arasındaki Korelasyon:", correlation)
print("----------------------------------------")

#  Bireysel ve ticari müşterileri filtreleme
bireysel_musteriler = ['BA', 'BB', 'O']
ticari_musteriler = ['K', 'TA', 'TB', 'SB', 'SC']

# Her bir müşteri tipinin karşılaştırılması
for musteri_tipi in ['Bireysel Müşteri', 'Ticari Müşteri']:
    if musteri_tipi == 'Bireysel Müşteri':
        musteri_df = df[df['Segment'].isin(bireysel_musteriler)]
    else:
        musteri_df = df[df['Segment'].isin(ticari_musteriler)]
        
for musteri_tipi in ['Bireysel Müşteri', 'Ticari Müşteri']:
    if musteri_tipi == 'Bireysel Müşteri':
        musteri_betscore_df = betscore_df[betscore_df['Segment'].isin(bireysel_musteriler)]
    else:
        musteri_betscore_df = betscore_df[betscore_df['Segment'].isin(ticari_musteriler)]

    
    gini_betscore = roc_auc_score(musteri_betscore_df['Default_Flag'], musteri_betscore_df['BET_Score'])
    gini_bankscore = roc_auc_score(musteri_df['Default_Flag'], musteri_df['Bank_Score'])

    print(musteri_tipi + " için BET Skor Kartının Performansı (Gini Katsayısı):", gini_betscore)
    print(musteri_tipi + " için Banka Skor Kartının Performansı (Gini Katsayısı):", gini_bankscore)
    print("----------------------------------------")

    
ticari_segmentler = ['K', 'TA', 'TB', 'SB', 'SC']
gini_betscore_ticari = {}
gini_bankscore_ticari = {}

for segment in ticari_segmentler:
    segment_df = df[df['Segment'] == segment]
    bet_segment_df = bet_df[bet_df['Segment'] == segment]
    
    if len(segment_df) > 0 and len(set(segment_df['Default_Flag'])) > 1:
        gini_betscore_segment = roc_auc_score(bet_segment_df['Default_Flag'], bet_segment_df['BET_Score'])
        gini_bankscore_segment = roc_auc_score(segment_df['Default_Flag'], segment_df['Bank_Score'])
        
        gini_betscore_ticari[segment] = gini_betscore_segment
        gini_bankscore_ticari[segment] = gini_bankscore_segment
        
        print(f"Segment: {segment}")
        print("Ticari Müşteriler için BET Skor Kartının Performansı (Gini Katsayısı):", gini_betscore_segment)
        print("Ticari Müşteriler için Banka Skor Kartının Performansı (Gini Katsayısı):", gini_bankscore_segment)
        print("----------------------------------------")
    else:
        print(f"Segment: {segment}")
        print("Bu segmentte yeterli müşteri yok.")
        print("----------------------------------------")


# Bireysel müşterileri segment ve ürün bazında filtreleme
bireysel_segmentler = ['BA', 'BB', 'O']
gini_betscore_bireysel = {}
gini_bankscore_bireysel = {}

for segment in bireysel_segmentler:
    segment_df = df[df['Segment'] == segment]
    bet_segment_df = bet_df[bet_df['Segment'] == segment]
    
    for product_type in range(6):
        product_df = segment_df[segment_df['Product_Type'] == product_type]
        bet_product_df = bet_segment_df[bet_segment_df['Product_Type'] == product_type]
        
        if len(product_df) > 0 and len(set(product_df['Default_Flag'])) > 1:
            gini_betscore_product = roc_auc_score(bet_product_df['Default_Flag'], bet_product_df['BET_Score'])
            gini_bankscore_product = roc_auc_score(product_df['Default_Flag'], product_df['Bank_Score'])
            
            if segment not in gini_betscore_bireysel:
                gini_betscore_bireysel[segment] = {}
                gini_bankscore_bireysel[segment] = {}
            
            gini_betscore_bireysel[segment][product_type] = gini_betscore_product
            gini_bankscore_bireysel[segment][product_type] = gini_bankscore_product
            
            print(f"Segment: {segment} - Product Type: {product_type}")
            print("Bireysel Müşteriler için BET Skor Kartının Performansı (Gini Katsayısı):", gini_betscore_product)
            print("Bireysel Müşteriler için Banka Skor Kartının Performansı (Gini Katsayısı):", gini_bankscore_product)
            print("----------------------------------------")
        else:
            print(f"Segment: {segment} - Product Type: {product_type}")
            print("Bu segment ve ürün kategorisi kombinasyonunda yeterli müşteri yok.")
            print("----------------------------------------")

Genel BET Skor Kartının Performansı (Gini Katsayısı): 0.7805141773097245
Genel Banka Skor Kartının Performansı (Gini Katsayısı): 0.7557646665446525
----------------------------------------
Banka Skoru ve BET Skoru Arasındaki Korelasyon: 0.8160247648875897
----------------------------------------
Bireysel Müşteri için BET Skor Kartının Performansı (Gini Katsayısı): 0.769693031366133
Bireysel Müşteri için Banka Skor Kartının Performansı (Gini Katsayısı): 0.7690981053608561
----------------------------------------
Ticari Müşteri için BET Skor Kartının Performansı (Gini Katsayısı): 0.7971751345079433
Ticari Müşteri için Banka Skor Kartının Performansı (Gini Katsayısı): 0.7690981053608561
----------------------------------------
Segment: K
Ticari Müşteriler için BET Skor Kartının Performansı (Gini Katsayısı): 0.963963963963964
Ticari Müşteriler için Banka Skor Kartının Performansı (Gini Katsayısı): 0.9801324503311258
----------------------------------------
Segment: TA
Ticari Müşteriler içi

##### 1- BET Skor kartının daha iyi performans istatistikleri sunduğu ürün türleri ve segmentler: TA, TB, SB, SC Segmentindeki Ticari Müşteriler, Ürün tipi 1,3,4,5 için BA Segmentindeki Bireysel Müşteriler, Ürün tipi 2,4,5 için BB Segmentindeki Bireysel Müşteriler, Ürün tipi 1,2,3,5 için O Segmentindeki Bireysel Müşteriler 

##### 2- Satıcı skoru ve Banka skoru arasındaki korelasyonu incelediğimizde %81'in üzerinde oldukça güçlü ve açıklayıcı bir korelasyona sahip olduğunu gözlemledik. Bu Satıcı skor kartının model süreçlerine katkı sağlayabileceğini ön görebiliyoruz.

##### 3- Elimizdeki veri seti özelinde yaptığımız analizlere dayanarak Satıcı skor kartının hem bireysel, hem ticari müşteriler için birçok segment ve ürün tipinde daha iyi sonuç verdiğini gözlemledik. Skor kartları arasındaki korelasyonun da yüksek olduğu göz onunda bulundurularak bankanın bu skor kartını satın almasını tavsiye edebiliriz.